In [1]:
import hid
import struct
import time
# [修复1] 导入 pynput 替代 pyautogui
from pynput.mouse import Button, Controller

# --- PyAutoGUI 的设置不再需要 ---

class XboxController:
    # --- [最终修正] 根据你的精确报文数据得出的按钮映射 ---
    # 你的控制器将按钮和DPAD放在了同一个字节 [4]
    # bit 0-3 是 DPAD, bit 4-7 是 A,B,X,Y
    BUTTON_MAP = {
        4: "A",
        5: "B",
        6: "X",
        7: "Y",
    }
    # 肩键等在另一个字节 [5]
    BUTTON_MAP_2 = {
        # 你的报文里，DPAD似乎不在这里，而是在字节4的低4位
        # 如果需要，可以重新映射。根据你的代码，这里是肩键和摇杆按键
        # 0: "DPAD_UP",  <-- 根据你的代码，这部分可能需要调整或确认
        # 1: "DPAD_DOWN",
        # 2: "DPAD_LEFT",
        # 3: "DPAD_RIGHT",
        4: "LB",
        5: "RB",
        6: "LS", # Left Stick Click
        7: "RS", # Right Stick Click
    }

    def __init__(self, vendor_id=0x045E, product_id=0x0B12):
        self.device = None
        try:
            self.device = hid.device()
            self.device.open(vendor_id, product_id)
            self.device.set_nonblocking(True)
            print("Connected:", self.device.get_manufacturer_string(),
                  self.device.get_product_string())
        except OSError as e:
            print(f"Error opening device: {e}")
            self.device = None

    def close(self):
        if self.device:
            self.device.close()

    def read(self):
        if not self.device:
            return None
            
        # 设定一个超时，避免在某些平台上 read() 即使在非阻塞模式下也等待
        data = self.device.read(64, timeout_ms=1) 
        if not data or len(data) < 18:
            return None

        raw = bytes(data)

        # ----- [关键] 根据你的报文进行的精确解析 -----
        
        buttons1_raw = raw[4]
        buttons2_raw = raw[5]

        lt, rt = struct.unpack_from("<HH", raw, 6)
        lx, ly, rx, ry = struct.unpack_from("<hhhh", raw, 10)

        buttons = self._decode_buttons(buttons1_raw, self.BUTTON_MAP)
        buttons.update(self._decode_buttons(buttons2_raw, self.BUTTON_MAP_2))

        return {
            "buttons": buttons,
            "lt_norm": lt / 1023.0,
            "rt_norm": rt / 1023.0,
            "lx_norm": self._normalize_axis(lx),
            "ly_norm": self._normalize_axis(ly),
            "rx_norm": self._normalize_axis(rx),
            "ry_norm": self._normalize_axis(ry),
        }

    def _decode_buttons(self, bitmask, button_map):
        return {name: bool(bitmask & (1 << bit)) for bit, name in button_map.items()}

    def _normalize_axis(self, v):
        v = int(v)
        # 稍微扩大分母，避免因为硬件差异导致无法达到完美的 1.0 或 -1.0
        if v < 0: return max(-1.0, v / 32768.0)
        else: return min(1.0, v / 32767.0)


if __name__ == "__main__":
    SCROLL_SPEED = 1 # pynput的滚动单位更大，所以值需要调小
    MOUSE_SENSITIVITY = 25 
    DEADZONE = 0.15

    try:
        xbox = XboxController() 
        if not xbox.device:
            raise OSError("Controller not found or could not be opened.")
        
        # [修复1] 初始化 pynput 的鼠标控制器
        mouse = Controller()

        print("\nController decoded successfully! Mouse control is active.")
        print("A = Left Click | B = Right Click | LB/RB = Scroll | Ctrl+C to exit.")
        print("-" * 50)

        a_button_pressed = False
        b_button_pressed = False
        
        # [修复2] 用于节流调试输出的变量
        last_print_time = time.time()
        
        while True:
            state = xbox.read()
            if state:
                # --- 鼠标移动 ---
                lx = state['lx_norm']
                ly = state['ly_norm']
                
                if abs(lx) < DEADZONE: lx = 0
                if abs(ly) < DEADZONE: ly = 0
                
                if lx != 0 or ly != 0:
                    # 使用立方曲线来获得更好的操控感（中心慢，边缘快）
                    x_move = (lx ** 3) * MOUSE_SENSITIVITY
                    # 你的报文向上推摇杆是正值，而屏幕坐标系向下是正值，所以需要一个负号
                    y_move = -(ly ** 3) * MOUSE_SENSITIVITY
                    # [修复1] 使用 pynput.mouse.move，响应更快
                    mouse.move(x_move, y_move)
                
                # --- 鼠标点击 ---
                if state['buttons'].get('A') and not a_button_pressed:
                    # [修复1] 使用 pynput.mouse.press
                    mouse.press(Button.left)
                    a_button_pressed = True
                elif not state['buttons'].get('A') and a_button_pressed:
                    # [修复1] 使用 pynput.mouse.release
                    mouse.release(Button.left)
                    a_button_pressed = False
                    
                if state['buttons'].get('B') and not b_button_pressed:
                    mouse.press(Button.right)
                    b_button_pressed = True
                elif not state['buttons'].get('B') and b_button_pressed:
                    mouse.release(Button.right)
                    b_button_pressed = False

                # --- 鼠标滚动功能 ---
                if state['buttons'].get('RB'):
                    # [修复1] 使用 pynput.mouse.scroll (dx, dy)
                    mouse.scroll(0, SCROLL_SPEED) # (水平, 垂直)
                
                if state['buttons'].get('LB'):
                    mouse.scroll(0, -SCROLL_SPEED)
                
                # [修复2] 节流调试输出，每0.1秒最多打印一次
                current_time = time.time()
                if current_time - last_print_time > 0.1:
                    pressed_buttons = sorted([name for name, pressed in state["buttons"].items() if pressed])
                    print(
                        f"Stick:({state['lx_norm']:.2f}, {state['ly_norm']:.2f}) "
                        f"LT:{state['lt_norm']:.2f} RT:{state['rt_norm']:.2f} "
                        f"Buttons: {pressed_buttons}      ", end='\r'
                    )
                    last_print_time = current_time

            # [修复3] 移除 time.sleep()。循环将由 hid.read() 的数据到达率自然控制
            # time.sleep(0.001) # <- 已移除

    except OSError as e:
        print(f"\nError: {e}")
    except KeyboardInterrupt:
        print("\nExiting.")
    finally:
        if 'xbox' in locals() and xbox:
            xbox.close()

Connected: Microsoft Controller

Controller decoded successfully! Mouse control is active.
A = Left Click | B = Right Click | LB/RB = Scroll | Ctrl+C to exit.
--------------------------------------------------
Stick:(-0.04, -0.02) LT:0.00 RT:0.00 Buttons: []         
Exiting.
